In [9]:
# Import Modules & Dependencies
import pandas as pd
from datetime import datetime
import getpass
import os

In [ ]:
# p = getpass.getpass(prompt="Password: ")
# rds_connection_string = f"postgres:{p}@localhost:5432/trimet_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
# Load Weather Data From CSV
weather_data_csv = os.path.join("raw_data","weather_data.csv")

# Store Data in a DataFrame
weather_data = pd.read_csv(weather_data_csv)

In [11]:
weather_data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,rain_1h,...,rain_6h,rain_12h,rain_24h,rain_today,snow_1h,snow_3h,snow_6h,snow_12h,snow_24h,snow_today
0,1483228800,2017-01-01 00:00:00 +0000 UTC,-28800,Beaverton,45.487172,-122.80378,35.64,33.80,37.40,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1483232400,2017-01-01 01:00:00 +0000 UTC,-28800,Beaverton,45.487172,-122.80378,35.47,33.80,37.40,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1483236000,2017-01-01 02:00:00 +0000 UTC,-28800,Beaverton,45.487172,-122.80378,34.70,33.08,35.96,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1483239600,2017-01-01 03:00:00 +0000 UTC,-28800,Beaverton,45.487172,-122.80378,36.03,35.06,37.40,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1483243200,2017-01-01 04:00:00 +0000 UTC,-28800,Beaverton,45.487172,-122.80378,34.14,32.00,35.96,NaN,...,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN


In [12]:
# Create a New DataFrame With Only the Necessary Columns
weather_data = weather_data[['dt_iso', 'rain_1h', 'snow_1h']]

In [13]:
# Simplify Column Names
weather_data = weather_data.rename(columns={'dt_iso':'date', 'rain_1h':'rain', 'snow_1h':'snow'})

In [14]:
# Fill N/A Values With a 0 For Future Calculations
weather_data = weather_data.fillna(0)

In [15]:
# Loop Through All Rows in the Date Column and Convert the Dates to a Datetime Data Type
for i in weather_data['date']:

    weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y-%m-%d %H:%M:%S %z UTC')

In [16]:
# Group Weather Data by Month, Regardless of City/Location
weather_data = weather_data.groupby(pd.Grouper(key='date', freq='M')).sum()

In [17]:
# Create a Total_Precip Column and Convert Total_Precip From MM to Inches
weather_data['total_precip'] = (weather_data['rain'] + weather_data['snow']) * 0.0394

In [18]:
# Divide Total_Precip by 6 to Account For the Six Cities in the Dataset and Obtain an Avg_Precip Value
weather_data['avg_precip'] = weather_data['total_precip'] / 6

In [19]:
# Format Data so That Every Value Only has Two Decimal Points
weather_data['rain'] = weather_data['rain'].map("{:.2f}".format)
weather_data['snow'] = weather_data['snow'].map("{:.2f}".format)
weather_data['total_precip'] = weather_data['total_precip'].map("{:.2f}".format)
weather_data['avg_precip'] = weather_data['avg_precip'].map("{:.2f}".format)

In [20]:
# Reset Index After Groupby
weather_data.reset_index(inplace=True)

In [21]:
weather_data.head()

,date,rain,snow,total_precip,avg_precip
0,2017-01-31 00:00:00+00:00,234.41,136.92,14.63,2.44
1,2017-02-28 00:00:00+00:00,1000.67,36.97,40.88,6.81
2,2017-03-31 00:00:00+00:00,594.85,4.26,23.60,3.93
3,2017-04-30 00:00:00+00:00,420.50,0.00,16.57,2.76
4,2017-05-31 00:00:00+00:00,179.12,0.00,7.06,1.18


In [22]:
# Add in column to capture the Month Year as a string
weather_data["date_str"] = weather_data["date"].dt.strftime("%b %Y")

In [23]:
# Add in column for month
weather_data["month"] = weather_data["date_str"].astype(str).str.slice(0,3)

# Add in column for year
weather_data["year"] = weather_data["date"].dt.year

# Preview table
weather_data.head()

,date,rain,snow,total_precip,avg_precip,date_str,month,year
0,2017-01-31 00:00:00+00:00,234.41,136.92,14.63,2.44,Jan 2017,Jan,2017
1,2017-02-28 00:00:00+00:00,1000.67,36.97,40.88,6.81,Feb 2017,Feb,2017
2,2017-03-31 00:00:00+00:00,594.85,4.26,23.60,3.93,Mar 2017,Mar,2017
3,2017-04-30 00:00:00+00:00,420.50,0.00,16.57,2.76,Apr 2017,Apr,2017
4,2017-05-31 00:00:00+00:00,179.12,0.00,7.06,1.18,May 2017,May,2017


In [24]:
# Create file path for exported table
file_path = os.path.join("static", "data", "final_weather_data.json")

# Export table as JSON file
weather_data.to_json(file_path, orient="table")

In [14]:
# weather_data.to_csv("static/data/weather_data.csv", index=False, header=True)

In [ ]:
# weather_data.to_sql('weather_data', con=engine, if_exists="append", index=False)